# Fed vs. fasted: comparing RSNs in giant Mappers

Let's recreate Figure 4 in Poldrack et al. 2015, Nature Communications, using Mapper!

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import *

In [3]:
%matplotlib inline

## Load data

In [4]:
sessions = load_scrubbed(n_sessions=10,  merge=True,#False,
    apply_tmask=True,
    apply_rmask=True, rmask_kwds=dict(n=5, minor=False),
    zscore=False
    )

INFO:load_data:load_scrubbed(**{'n_sessions': 10, 'merge': True, 'apply_tmask': True, 'apply_rmask': True, 'rmask_kwds': {'n': 5, 'minor': False}, 'zscore': False})
INFO:load_data:Loading data...
INFO:root:  [+] session: 0, file: sub014.txt
INFO:root:      keeping: 505 (time points)
INFO:root:      keeping: 437 (regions)
INFO:root:  [+] session: 1, file: sub015.txt
INFO:root:      keeping: 508 (time points)
INFO:root:      keeping: 437 (regions)
INFO:root:  [+] session: 2, file: sub016.txt
INFO:root:      keeping: 508 (time points)
INFO:root:      keeping: 437 (regions)
INFO:root:  [+] session: 3, file: sub017.txt
INFO:root:      keeping: 504 (time points)
INFO:root:      keeping: 437 (regions)
INFO:root:  [+] session: 4, file: sub018.txt
INFO:root:      keeping: 508 (time points)
INFO:root:      keeping: 437 (regions)
INFO:root:  [+] session: 5, file: sub019.txt
INFO:root:      keeping: 513 (time points)
INFO:root:      keeping: 437 (regions)
INFO:root:  [+] session: 6, file: sub020.t

### Clean data

Load temporal masks and remove TRs with excessive motion. Additionally, remove parcels that do not belong to major RSNs.

In [5]:
splits = get_data_splits(sessions, zscore=True, groupby='day_of_week', groups={2:'fasted', 4:'fed'}, density=0.02)

print('splits: ', splits.keys())

            1.0 => day_of_week_0    x.shape: (1517, 437)  RSN.shape: (1517, 5)
            2.0 => fasted           x.shape: (1532, 437)  RSN.shape: (1532, 5)
            4.0 => fed              x.shape: (1856, 437)  RSN.shape: (1856, 5)
splits:  dict_keys(['day_of_week_0', 'fasted', 'fed'])


## Generate $MAPPER$ graphs

### Start local server for visualization

In [6]:
%%bash --bg

python3 -m http.server 8800

Starting job # 0 in a separate thread.


### Run Mapper in temporal space - create one super graph for both fed and fasted

#### (split = fed)

In [24]:
# Run Mapper/DyNeuSR
print('Running mapper on fed data...')
results_fed = run_mapper(splits.fed.X, splits.fed.RSN, verbose=1, cover=km.Cover(10,0.3))

Running mapper on fed data...
KeplerMapper()
Mapping on data shaped (1856, 437) using lens shaped (1856, 2)

Creating 100 hypercubes.

Created 159 edges and 79 nodes in 0:00:00.325349.


label,group,value,row_count
Cingulo_opercular,0,38,1855
Frontoparietal_1,1,38,1855
Somatomotor,2,38,1855
Visual_2,3,38,1855
DMN,4,38,1855


In [31]:
# Visualize DyNeuGraph
results_fed['dG'].visualize(path_html='results/fed_super_n=10.html', show=False, reset=True, static=False, PORT=8800)

[Force Graph] http://localhost:8800/results/fed_super_n=10.html


DyNeuGraph()

##### Quick look (png)

![fed_super](./results/png/fed_super_n=10.png)

#### (split = fasted)

In [19]:
# Run Mapper/DyNeuSR
print('Running mapper on fasted data...')
results_fast = run_mapper(splits.fasted.X, splits.fasted.RSN, verbose=2, cover=km.Cover(10,0.3))

Running mapper on fasted data...
KeplerMapper()
..Composing projection pipeline of length 1:
	Projections: TSNE(angle=0.5, early_exaggeration=12.0, init='random', learning_rate=200.0,
   method='barnes_hut', metric='euclidean', min_grad_norm=1e-07,
   n_components=2, n_iter=1000, n_iter_without_progress=300, perplexity=50,
   random_state=0, verbose=0)
	Distance matrices: False
	Scalers: MinMaxScaler(copy=True, feature_range=(0, 1))
..Projecting on data shaped (1532, 437)

..Projecting data using: 
	TSNE(angle=0.5, early_exaggeration=12.0, init='random', learning_rate=200.0,
   method='barnes_hut', metric='euclidean', min_grad_norm=1e-07,
   n_components=2, n_iter=1000, n_iter_without_progress=300, perplexity=50,
   random_state=0, verbose=1)

[t-SNE] Computing 151 nearest neighbors...
[t-SNE] Indexed 1532 samples in 0.017s...
[t-SNE] Computed neighbors for 1532 samples in 1.679s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1532
[t-SNE] Computed conditional probabili

label,group,value,row_count
Cingulo_opercular,0,31,1531
Frontoparietal_1,1,31,1531
Somatomotor,2,31,1531
Visual_2,3,31,1531
DMN,4,31,1531


In [30]:
# Visualize DyNeuGraph
results_fast['dG'].visualize(path_html='results/fast_super_n=10.html', show=False, reset=True, static=False, PORT=8800)

[Force Graph] http://localhost:8800/results/fast_super_n=10.html


DyNeuGraph()

##### Quick look (png)


![fast_super](./results/png/fast_super_n=10.png)

### Run Mapper in anatomical space - create one super graph for both fed and fasted